# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [52]:
def import_python_packages():
    # Import Python packages 
    import pandas as pd
    import cassandra
    import re
    import os
    import glob
    import numpy as np
    import json
    import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
def create_list_of_files(filepath):

    """
    Description: This function is responsible for listing the files in a directory,
    
    Arguments:
        filepath: directory containing file to list

    Returns:
        file_path_list: list of files to process
    """

    # Create a for loop to create a list of files and collect each filepath
    all_files = []

    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files :
            all_files.append(os.path.abspath(f))
        
    # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*'))
        # print(file_path_list)

    return file_path_list

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
def process_csv_files(file_path_list, result_file_name):

    """
    Description: This function process csv files to create a smaller event data csv file \
    called event_datafile_full.csv that will be used to insert data \
    into the Apache Cassandra tables
    
    Arguments:
        file_path_list: list of files to process

    Returns:
        file_path_list: list of files to process
    """

    # initiating an empty list of rows that will be generated from each file
    full_data_rows_list = [] 
        
    # for every filepath in the file path list 
    for f in file_path_list:

    # reading csv file
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)
            
    # extracting each data row one by one and append it        
            for line in csvreader:
                #print(line)
                full_data_rows_list.append(line) 
                

    # creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
    # Apache Cassandra tables
    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open(result_file_name, 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

# Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
def cassandra_connection(hostname):

    """
    Description: This function makes a connection to a Cassandra instance
    
    Arguments:
        hostname: hostname of the Cassandra instance

    Returns:
        session: used to communicate with the Cassandra instance
    """

    from cassandra.cluster import Cluster

    try:
        cluster = Cluster([hostname])
        session = cluster.connect()
    except Exception as e:
        print (e)
    
    return session, cluster

#### Create Keyspace

In [ ]:
def create_set_keyspace(session):

    """
    Description: creates and set a new Keyspace (if it does not exist) in the Cassandra instance
    
    Arguments:
        session: used to communicate with the Cassandra instance

    Returns:
        None
    """

    try:
        session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : '1'}
        """)
    except Exception as e:
        print(e)

    try:
        session.set_keyspace('udacity')
    except Exception as e:
        print(e)

#### Set Keyspace

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [ ]:
def process_query1(event_data_file_name, session):

    """
    Description: creates table, import file in the table then do a select statement
    
    Arguments:
        result_file_name: file to import in the table

    Returns:
        None
    """

    # Create table ml_for_artist_song_songlength
    try:
        query = "CREATE TABLE IF NOT EXISTS ml_for_artist_song_songlength "
        query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length decimal, PRIMARY KEY (session_id, item_in_session))"
        session.execute(query)
    except Exception as e:
        print(e)
    
    # Import event data file in table
    with open(event_data_file_name, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            try:
                query = "INSERT INTO ml_for_artist_song_songlength (session_id, item_in_session, artist_name, song_title, song_length)"
                query = query + "VALUES (%s, %s, %s, %s, %s)"
                session.execute_async(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
            except Exception as e:
                print(e)

    #SELECT statement to display the result
    query = "SELECT artist_name, song_title, song_length FROM ml_for_artist_song_songlength WHERE session_id = 338 and item_in_session = 4"
    header = "Artist Name // Song Title // Song Length" 
    print(header)

    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)
    
    for row in rows:
        print(row.artist_name, " // " , row.song_title, " // " , row.song_length)

# Query2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
def process_query2(event_data_file_name, session):

    """
    Description: creates table, import file in the table then do a select statement
    
    Arguments:
        result_file_name: file to import in the table

    Returns:
        None
    """

    # Create table ml_for_artist_song_username
    query = "CREATE TABLE IF NOT EXISTS ml_for_artist_song_username "
    query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, PRIMARY KEY (user_id, session_id, item_in_session))"

    session.execute(query)

    # Import event data file in table
    with open(event_data_file_name, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            try:
                query = "INSERT INTO ml_for_artist_song_username (user_id, session_id, item_in_session, artist_name, song_title)"
                query = query + "VALUES (%s, %s, %s, %s, %s)"
                session.execute_async(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9]))
            except Exception as e:
                print(e)
    
    #SELECT statement to display the result
    query = "SELECT artist_name, song_title, item_in_session FROM ml_for_artist_song_username WHERE user_id = 10 and session_id = 182"

    header = "Item In Session // Artist Name // Song Title" 
    print(header)

    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        print(row.item_in_session, " // ", row.artist_name, " // " , row.song_title)

# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
def process_query3(event_data_file_name, session):

    """
    Description: creates table, import file in the table then do a select statement
    
    Arguments:
        result_file_name: file to import in the table

    Returns:
        None
    """

    # Create table ml_for_username
    query = "CREATE TABLE IF NOT EXISTS ml_for_username "
    query = query + "(song_title text, item_in_session int, session_id int, username text, PRIMARY KEY (song_title, session_id, item_in_session))"

    session.execute(query)
    with open(event_data_file_name, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            try:
                username = line[1] + " " + line[4]
                query = "INSERT INTO ml_for_username (song_title, item_in_session, session_id, username)"
                query = query + "VALUES (%s, %s, %s, %s)"
                session.execute_async(query, (line[9], int(line[3]), int(line[8]), username))
            except Exception as e:
                print(e)
                
    # SELECT statement to display the result
    query = "SELECT username FROM ml_for_username WHERE song_title = 'All Hands Against His Own'"

    header = "UserName"
    print(header)

    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        print(row.username)

# Drop the tables before closing out the sessions

In [55]:
def drop_table(table_name, session):
    ## TO-DO: Drop the table before closing out the sessions
    try:
        session.execute("DROP TABLE IF EXISTS " + table_name)
    except Exception as e:
        print(e)


In [56]:
def main():

    # Set variables
    filepath = os.getcwd() + '/event_data'
    event_data_file_name = "event_datafile_new.csv"
    hostname = "127.0.0.1"
    keyspace_name = "udacity"
    
    # Import python packages
    import_python_packages()

    # file processing part
    list_of_files = create_list_of_files(filepath)
    process_csv_files(list_of_files, event_data_file_name)

    # Cassandra part
    session, cluster = cassandra_connection(hostname)
    create_set_keyspace(session)

    #CQL queries
    process_query1(event_data_file_name, session)
    print("")

    process_query2(event_data_file_name, session)
    print("")

    process_query3(event_data_file_name, session)
    print("")

    # Drop tables
    drop_table("ml_for_artist_song_songlength", session)
    drop_table("ml_for_artist_song_username", session)
    drop_table("ml_for_username", session)
    session.execute("DROP KEYSPACE " + keyspace_name)

    # Close the session and cluster connection
    session.shutdown()
    cluster.shutdown()


if __name__ == "__main__":
    main()

Artist Name // Song Title // Song Length
Faithless  //  Music Matters (Mark Knight Dub)  //  495.3073

Item In Session // Artist Name // Song Title
0  //  Down To The Bone  //  Keep On Keepin' On
1  //  Three Drives  //  Greece 2000
2  //  Sebastien Tellier  //  Kilometer
3  //  Lonnie Gordon  //  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)

UserName
Sara Johnson
Jacqueline Lynch
Tegan Levine

